In [1]:
!wget https://market1501.s3-us-west-2.amazonaws.com/dataset.zip
!unzip -q dataset.zip -d dataset

!rm -rf /content/deeplearning_unitn
!git clone https://github.com/deayalar/deeplearning_unitn.git

!nvidia-smi

%cd /content/deeplearning_unitn

--2021-06-16 22:02:21--  https://market1501.s3-us-west-2.amazonaws.com/dataset.zip
Resolving market1501.s3-us-west-2.amazonaws.com (market1501.s3-us-west-2.amazonaws.com)... 52.218.236.129
Connecting to market1501.s3-us-west-2.amazonaws.com (market1501.s3-us-west-2.amazonaws.com)|52.218.236.129|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82925180 (79M) [application/zip]
Saving to: ‘dataset.zip.2’

dataset.zip.2       100%[===================>]  79.08M  29.3MB/s    in 2.7s    

2021-06-16 22:02:24 (29.3 MB/s) - ‘dataset.zip.2’ saved [82925180/82925180]

replace dataset/annotations_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
Cloning into 'deeplearning_unitn'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 135 (delta 71), reused 102 (delta 43), pack-reused 0
Receiving objects: 100% (135/135), 5.37 MiB | 13.04 MiB/s, done.
Resolving deltas: 100% (71

In [2]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

from cost_functions import cross_entropy
import cost_functions
from evaluation import Evaluator
from my_datasets.reid_dataset import Market1501
from utils.split_data import ValidationSplitter, TrainingSplitter
from models.reid_model import ReIdModel, FinetunedModel
from sklearn.model_selection import train_test_split

In [3]:
config = dict(
    wandb = False,
    device = "auto", # Select an specific device None to select automatically
    train_root = "/content/dataset/train",
    test_root = "/content/dataset/test",
    queries_root = "/content/dataset/queries",
    attributes_file = "/content/dataset/annotations_train.csv",
    # train_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/train",
    # test_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/test",
    # queries_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/queries",
    # attributes_file = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/annotations_train.csv",
    dataset="Market1501",
    backbone = "resnet18",
    loss = cross_entropy(),
    split = dict(
        full_training_size = 0.75,
        train_size = 0.8
    ),
    compose = dict(
        resize_h = 224,
        resize_w = 224
    ),
    epochs=1,
    training_batch_size=32,
    validation_batch_size=32,
    learning_rate=0.01,
    weight_decay=0.000001, 
    momentum=0.9, 
    mAP_rank=10)

In [4]:
def split_data(config):
    print('split_data OK!')
    splitter = ValidationSplitter(train_root=config["train_root"], 
                                  test_root=config["test_root"], 
                                  queries_root=config["queries_root"])
    train_set, val_set, val_queries = splitter.split(train_size=config["split"]["full_training_size"],
                                                     random_seed=42)
    return train_set, val_set, val_queries
train_set, val_set, val_queries = split_data(config)

split_data OK!
Extract queries proportion: 0.11
Identities in train set: 563
Identities in validation set: 188
Train set size: 9721
Validation set size: 2908
Number of validation queries: 360


In [5]:
from my_datasets.reid_dataset import Market1501

#Create pytorch Datasets 
composed = transforms.Compose([transforms.Resize((config["compose"]["resize_h"], 
                                                  config["compose"]["resize_w"])),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                    std=[0.229, 0.224, 0.225])])

train_dataset = Market1501(root_dir=config["train_root"],
                        attributes_file=config["attributes_file"],
                        #full_train_set=full_train_set,
                        images_list=train_set,
                        transform=composed)

In [6]:
train_loader = torch.utils.data.DataLoader(train_dataset, 
                                            batch_size=config["training_batch_size"], 
                                            shuffle=True, 
                                            # num_workers=8
                                            num_workers=4)
                                            
# val_loader = torch.utils.data.DataLoader(val_dataset, 
#                                             batch_size=config["validation_batch_size"], 
#                                             shuffle=False, 
#                                             num_workers=8)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import models

PRETRAINED_MODELS = {
    "resnet18": { "load": lambda : models.resnet18(pretrained=True), "feature_size": 512},
    "resnet50": { "load": lambda : models.resnet50(pretrained=True), "feature_size": 2048}
    # More pretrained models here e.g. alexnet, vgg16, etc
}

class FinetunedModel(nn.Module):
    def __init__(self, architecture, n_classes):
        super(FinetunedModel, self).__init__()
        self.architecture = architecture

        self.backbone = PRETRAINED_MODELS[architecture]["load"]()
        self.feature_size = PRETRAINED_MODELS[architecture]["feature_size"]
        print(f"Backbone feature size: {self.feature_size}")
        self.finetune(self.backbone, n_classes)

    def finetune(self, model, n_classes):
        model_name = model.__class__.__name__
        if model_name.lower().startswith("resnet"):
            self.features = nn.Sequential(*list(model.children())[:-1])
            self.classifier = nn.Sequential(
                nn.Linear(self.feature_size, n_classes)
            )

    def forward(self, x, get_features=False):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        if get_features:
            return x
        x = self.classifier(x)
        return x


class ReIdModel(nn.Module):
    "Model based on LeNet for person re-identification"
    def __init__(self, n_classes):
        super().__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5, stride=1),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.BatchNorm2d(120),
            nn.ReLU()
        )

        self.classifier = nn.Sequential(
            nn.Linear(in_features=7 * 7 * 120, out_features=120),
            nn.BatchNorm1d(120),
            nn.ReLU(),
            nn.Linear(in_features=120, out_features=84),
            nn.BatchNorm1d(84),
            nn.ReLU(),
            nn.Linear(in_features=84, out_features=n_classes),
        )


    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [35]:
'''
Let's take the backbone. weights trained. 

'''


def initialize_resnet18(num_classes):
    model_conv = torchvision.models.resnet18(pretrained=True)

    num_ftrs = model_conv.fc.in_features
    model_conv.fc = nn.Linear(num_ftrs, num_classes)
    return model_conv


# def initialize_alexnet(num_classes):
#     # load the pre-trained Alexnet
#     alexnet = torchvision.models.alexnet(pretrained=True)
  
#     # get the number of neurons in the penultimate layer
#     in_features = alexnet.classifier[6].in_features
    
#     # re-initalize the output layer
#     alexnet.classifier[6] = torch.nn.Linear(in_features=in_features, 
#                                             out_features=num_classes)
    
#     return alexnet


def get_cost_function():
    cost_function = torch.nn.CrossEntropyLoss()
    return cost_function

def get_optimizer(model, lr, wd, momentum):
  
    # we will create two groups of weights, one for the newly initialized layer
    # and the other for rest of the layers of the network
    
    final_layer_weights = []
    rest_of_the_net_weights = []
    
    # we will iterate through the layers of the network
    for name, param in model.named_parameters():
        if name.startswith('classifier.6'):
            final_layer_weights.append(param)
        else:
            rest_of_the_net_weights.append(param)
    
    # so now we have divided the network weights into two groups.
    # We will train the final_layer_weights with learning_rate = lr
    # and rest_of_the_net_weights with learning_rate = lr / 10
    
    optimizer = torch.optim.SGD([
        {'params': rest_of_the_net_weights},
        {'params': final_layer_weights, 'lr': lr}
    ], lr=lr / 10, weight_decay=wd, momentum=momentum)
    
    return optimizer


def train(net,data_loader,optimizer,cost_function, device='cuda:0'):
    samples = 0.
    cumulative_loss = 0.
    cumulative_accuracy = 0.

    
    net.train() # Strictly needed if network contains layers which has different behaviours between train and test
    # for batch_idx, (inputs, targets) in enumerate(data_loader):
    for  (batch_idx,targets) in enumerate(data_loader):
        # print('inputs: ',targets[0].size(),'labels: ', targets[1].size())

        t1 =targets[1].to(device)

        # Load data into GPU
        inputs = targets[0].to(device)
        # inputs = targets[0]
        targets = targets[2].to(device)
        # targets = targets[2]
        
        
        # Forward pass
        outputs = net(inputs)
        # print('outputs: ',outputs.size())

        # Apply the loss
        # loss = cost_function(outputs,targets)
        loss = cost_function(outputs, torch.max(targets, 1)[1])
        # loss = criterion(outputs, torch.max(labels, 1)[1])
        # Backward pass
        loss.backward()
        
        # Update parameters
        optimizer.step()
        
        # Resets the gradients
        optimizer.zero_grad()

        # Better print something, no?
        samples+=inputs.shape[0]
        cumulative_loss += loss.item()
        _, predicted = outputs.max(1)
        # try:
        #   print(predicted.size())
        #   print(t1.size())
        #   print(predicted.eq(t1))
        # except:
        #   print(targets[0].size())
        #   print(predicted.eq(targets[0]))

        cumulative_accuracy += predicted.eq(t1).sum().item()
        # cumulative_accuracy += predicted.eq(targets[1]).sum()

    return cumulative_loss/samples, cumulative_accuracy/samples*100

In [36]:
def main(train_loader,
        batch_size=128, 
         device='cuda:0', 
         learning_rate=0.001, 
         weight_decay=0.000001, 
         momentum=0.9, 
         epochs=50, 
         num_classes=27
         ):
  
  #   writer = SummaryWriter(log_dir="runs/exp1")

  # Instantiates dataloaders
  #   train_loader, test_loader = get_data(batch_size=batch_size, img_root=img_root)
  
  # Instantiates the model
  net = initialize_resnet18(num_classes=num_classes).to(device)
  
  # Instantiates the optimizer
  optimizer = get_optimizer(net, learning_rate, weight_decay, momentum)
  
  # Instantiates the cost function
  cost_function = get_cost_function()

  #   print('Before training:')
#   train_loss, train_accuracy = test(net, train_loader, cost_function)
#   test_loss, test_accuracy = test(net, test_loader, cost_function)

#   print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
#   print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
#   print('-----------------------------------------------------')
  
  # Add values to plots
#   writer.add_scalar('Loss/train_loss', train_loss, 0)
#   writer.add_scalar('Loss/test_loss', test_loss, 0)
#   writer.add_scalar('Accuracy/train_accuracy', train_accuracy, 0)
#   writer.add_scalar('Accuracy/test_accuracy', test_accuracy, 0)

  for e in range(epochs):
    train_loss, train_accuracy = train(net, train_loader, optimizer, cost_function)
    # test_loss, test_accuracy = test(net, test_loader, cost_function)
    print('Epoch: {:d}'.format(e+1))
    print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
    # print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
    print('-----------------------------------------------------')
    
    # Add values to plots
    # writer.add_scalar('Loss/train_loss', train_loss, e + 1)
    # writer.add_scalar('Loss/test_loss', test_loss, e + 1)
    # writer.add_scalar('Accuracy/train_accuracy', train_accuracy, e + 1)
    # writer.add_scalar('Accuracy/test_accuracy', test_accuracy, e + 1)

  print('After training:')
  train_loss, train_accuracy = test(net, train_loader, cost_function)
#   test_loss, test_accuracy = test(net, test_loader, cost_function)

  print('\t Training loss {:.5f}, Training accuracy {:.2f}'.format(train_loss, train_accuracy))
#   print('\t Test loss {:.5f}, Test accuracy {:.2f}'.format(test_loss, test_accuracy))
  print('-----------------------------------------------------')

  # Closes the logger
#   writer.close()

In [37]:
main(train_loader,
        batch_size=32,
        device='cuda:0', 
         learning_rate=0.001, 
         weight_decay=0.000001, 
         momentum=0.9, 
         epochs=50, 
         num_classes=27
         )

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 1
	 Training loss 0.00648, Training accuracy 0.16
-----------------------------------------------------


KeyboardInterrupt: ignored

# NOT HERE YET!

In [ ]:
from utils import image_utils
image_utils.imshow(train_dataset[10][0])
print(train_dataset[10])

In [ ]:
print(len(train_dataset.attr))
print(len(train_dataset.class_to_idx))

In [ ]:
from utils import image_utils
print(train_dataset.images_list[0])
print(image_utils.get_ids_from_images(train_dataset.images_list)[0])
image_utils.imshow(train_dataset[0][0])


In [ ]:
train_dataset.__init__(root_dir=config["train_root"],
                        attributes_file=config["attributes_file"], images_list=img_l).__getitem__()

In [ ]:
img_l = train_dataset.images_list

In [ ]:
train_dataset.__getitem__(int('0901'))

In [ ]:
train_dataset.__getitem__(int('0901'))

In [ ]:
idx = train_dataset.class_to_idx.keys()
# idx
train_dataset.__getitem__(int('0901'))

In [ ]:
train_dataset